In [74]:
import pandas as pd
import numpy as np

In [75]:
df = pd.read_csv('test.csv')
df.head()

,Game,Rally,Team,No.,Space,Action,Errors,Nothing,Score
0,21,25.0,USA,15,Y,JS_0,NaN,1.0,NaN
1,21,25.0,BRA,18,7,R_0,NaN,1.0,NaN
2,21,25.0,BRA,1,2,G_0,NaN,1.0,NaN
3,21,25.0,BRA,23,2,A_0,NaN,NaN,1.0
4,21,25.0,USA,8,8,D_0,1.0,NaN,NaN


index = 1069 的 Team = POL, 
index = 2364 的 Team = IRI

In [76]:
df.loc[2364, 'Team'] = 'IRI'
df.loc[1069, 'Team'] = 'POL'

In [77]:
df.to_csv('test2.csv', index=False)

In [78]:
for i, row in df[df['Game'] == 2].iterrows():
    if(row['Rally'] == 50):
        print(i, row)

2364 Game          2
Rally        50
Team        IRI
No.           2
Space         Y
Action     JS_0
Errors      NaN
Nothing     NaN
Score         1
Name: 2364, dtype: object
2365 Game         2
Rally       50
Team       POL
No.         17
Space        8
Action     R_0
Errors       1
Nothing    NaN
Score      NaN
Name: 2365, dtype: object


In [79]:
df.groupby('Team').count()

,Game,Rally,No.,Space,Action,Errors,Nothing,Score
Team,,,,,,,,
BRA,546,546,546,546,546,95,385,65
FRA,1207,1207,1207,1207,1207,202,822,183
IRI,550,550,550,550,550,107,373,68
ITA,1271,1271,1271,1271,1271,233,896,139
JAP,352,352,352,352,352,76,239,36
NED,532,532,532,532,532,90,381,61
POL,1264,1264,1264,1264,1264,220,890,154
UAS,119,119,119,119,119,32,73,14
USA,1306,1306,1306,1306,1306,233,901,171


In [80]:
player_list = []
for _, row in df.iterrows():
    player_list.append(str(row['Team']) + '_' + str(row['No.']))

In [81]:
df.insert(2, 'Player', player_list)

In [82]:
# df = df.drop('Team', axis=1) # Team 要留著後續切分不同 shot
df = df.drop('No.', axis=1)

In [86]:
df.groupby('Player').count()

,Game,Rally,Team,Space,Action,Errors,Nothing,Score
Player,,,,,,,,
BRA_1,112,112,112,112,112,12,100,0
BRA_11,4,4,4,4,4,1,3,0
BRA_12,23,23,23,23,23,3,20,0
BRA_14,23,23,23,23,23,2,21,0
BRA_15,4,4,4,4,4,2,2,0
...,...,...,...,...,...,...,...,...
USA_4,112,112,112,112,112,18,67,27
USA_5,85,85,85,85,85,17,53,15
USA_6,2,2,2,2,2,2,0,0


In [85]:
df.to_csv('new_test.csv', index=False)

打亂 sequence
* 但每個 shot 之間的三個動作應該是不能被打亂的？
* 所以是 by Game + Rally

In [186]:
from sklearn.utils import shuffle
import random as rd

In [187]:
rally_list = []
new_df = pd.DataFrame(columns=df.columns)
for _, rally in df.groupby(['Game', 'Rally']):
    curr_team = df.iloc[rally.index[0]]['Team']
    shuffle_list = []
    tmp_list = []
    for _, row in rally.iterrows():
        if(row['Team'] != curr_team):
            shuffle_list.append(tmp_list)
            tmp_list = []
            curr_team = row['Team']
        tmp_list.append(row.tolist())
    shuffle_list.append(tmp_list)
    ############## shuffle ########################
    shuffled_list = shuffle(shuffle_list, random_state=3)
    restore_list = []
    for i in shuffled_list:
        restore_list += i
    shuffled_df = pd.DataFrame(restore_list, columns=df.columns)
    ############## combine df #####################]
    rally_list.append(shuffled_df)

In [188]:
rd.shuffle(rally_list)
new_df = pd.DataFrame(columns=df.columns)

for rally in rally_list:
    new_df = pd.concat([new_df, rally])

In [189]:
new_df

,Game,Rally,Player,Team,Space,Action,Errors,Nothing,Score
0,7,20.0,ITA_7,ITA,8,R_1,NaN,1.0,NaN
1,7,20.0,ITA_6,ITA,2,G_1,NaN,1.0,NaN
2,7,20.0,ITA_19,ITA,2,A_0,NaN,NaN,1.0
3,7,20.0,FRA_9,FRA,Y,JS_0,NaN,1.0,NaN
4,7,20.0,FRA_9,FRA,5,D_0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4,11,23.0,NED_14,NED,3,A_0,NaN,1.0,NaN
5,11,23.0,ITA_5,ITA,L,JS_0,NaN,1.0,NaN
6,11,23.0,ITA_6,ITA,6,D_0,NaN,1.0,NaN
7,11,23.0,ITA_7,ITA,1,G_0,NaN,1.0,NaN


In [190]:
new_df.to_csv('permutation_df.csv', index=False)